In [1]:
import math

import geopandas as gpd
import pandas as pd
import salem

import xarray as xr

In [2]:
senamhi_sectors = salem.read_shapefile(
    "/data/users/grivera/Shapes/SENAMHI_REGIONS/sectores_climaticos.shp"
)
senamhi_sectors

,id,sector,geometry,min_x,max_x,min_y,max_y
0,0,CostaNorte,"POLYGON ((-80.27326 -3.40197, -80.25299 -3.377...",-81.333149,-78.292190,-9.144099,-3.377033
1,1,SierraNorteOccidental,"POLYGON ((-79.83215 -4.45875, -79.77838 -4.480...",-80.097126,-77.842515,-8.785606,-4.427574
2,2,CostaCentro,"POLYGON ((-78.59145 -9.14410, -78.54781 -9.103...",-78.601586,-76.082780,-13.288594,-8.779371
3,3,CostaSur,"POLYGON ((-76.23865 -13.28859, -76.21371 -13.2...",-76.389838,-70.077238,-18.327763,-13.145196
4,4,SierraCentralOccidental,"POLYGON ((-77.84251 -8.12785, -77.81134 -8.102...",-78.302322,-74.778176,-14.403040,-8.096675
5,7,SierraSurOccidental,"POLYGON ((-74.81870 -14.05702, -74.77506 -14.0...",-75.113290,-69.497414,-18.318411,-14.053899
6,5,SierraNorteOriental,"MULTIPOLYGON (((-79.38248 -4.80477, -79.34390 ...",-79.578870,-76.615455,-8.572848,-4.804771
7,6,SierraCentralOriental,"MULTIPOLYGON (((-76.72417 -8.53427, -76.61896 ...",-77.730291,-74.228503,-13.751518,-8.106417
8,8,SierraSurOriental,"MULTIPOLYGON (((-74.65285 -12.04653, -74.63181...",-74.894833,-68.960989,-17.280339,-11.993926
9,9,SelvaSurAlta,"MULTIPOLYGON (((-74.67126 -12.18067, -74.66775...",-74.950945,-68.841970,-14.518966,-11.285074


In [3]:
pisco = (
    xr.open_dataset("/data/users/grivera/PISCOPrecv2p1-unstable.nc", decode_times=False)
    .rename({"X": "lon", "Y": "lat", "T": "time"})
    .load()
)
pisco.time.attrs["calendar"] = "360_day"
pisco = xr.decode_cf(pisco).Prec.convert_calendar("standard", align_on="year")
pisco

<xarray.DataArray 'Prec' (time: 498, lat: 198, lon: 133)>
[13114332 values with dtype=float32]
Coordinates:
  * lon      (lon) float32 -81.25 -81.15 -81.05 -80.95 ... -68.25 -68.15 -68.05
  * time     (time) datetime64[ns] 1981-01-16 1981-02-16 ... 2022-06-17
  * lat      (lat) float32 0.95 0.85 0.75 0.65 ... -18.45 -18.55 -18.65 -18.75
Attributes:
    pointwidth:      1.0
    expires:         1659312000
    long_name:       Precipitation
    units:           mm/month
    maxncolor:       254
    ncolor:          254
    colormap:        [null 16777215 16777215 16777215 [16777215 3] 14155730 [...
    colorscalename:  precip_colors
    scale_min:       0.0
    CS:              0

In [4]:
pisco_clim = (
    pisco.sel(time=slice("1981-10-01", "2016-10-01")).groupby("time.month").mean("time")
)
pisco_clim

<xarray.DataArray 'Prec' (month: 12, lat: 198, lon: 133)>
array([[[        nan,         nan,         nan, ..., 251.15376  ,
         183.61168  , 179.36876  ],
        [        nan,         nan,         nan, ..., 276.2781   ,
         216.03697  , 210.02179  ],
        [        nan,         nan,         nan, ..., 313.21503  ,
         262.69818  , 240.14609  ],
        ...,
        [        nan,         nan,         nan, ...,  99.41221  ,
          99.35651  , 100.31437  ],
        [        nan,         nan,         nan, ..., 105.119865 ,
         105.14163  ,  98.91334  ],
        [        nan,         nan,         nan, ...,  96.36325  ,
          88.84843  ,  82.965645 ]],

       [[        nan,         nan,         nan, ..., 244.40715  ,
         197.01471  , 192.1573   ],
        [        nan,         nan,         nan, ..., 263.53983  ,
         219.43385  , 228.31633  ],
        [        nan,         nan,         nan, ..., 289.4235   ,
         246.70209  , 236.06917  ],
...
        [        nan,         nan,         nan, ...,   1.876654 ,
           2.0272143,   2.2584147],
        [        nan,         nan,         nan, ...,   2.0495293,
           2.251868 ,   5.0499215],
        [        nan,         nan,         nan, ...,   4.9522204,
           5.625344 ,   5.6225576]],

       [[        nan,         nan,         nan, ..., 247.55344  ,
         398.65283  , 526.26196  ],
        [        nan,         nan,         nan, ..., 279.1794   ,
         367.8443   , 424.9211   ],
        [        nan,         nan,         nan, ..., 287.22342  ,
         305.35245  , 288.2079   ],
        ...,
        [        nan,         nan,         nan, ...,  28.309082 ,
          22.570148 ,  23.97255  ],
        [        nan,         nan,         nan, ...,  20.438408 ,
          20.250645 ,  26.477146 ],
        [        nan,         nan,         nan, ...,  22.174809 ,
          24.95753  ,  30.09798  ]]], dtype=float32)
Coordinates:
  * lon      (lon) float32 -81.25 -81.15 -81.05 -80.95 ... -68.25 -68.15 -68.05
  * lat      (lat) float32 0.95 0.85 0.75 0.65 ... -18.45 -18.55 -18.65 -18.75
  * month    (month) int64 1 2 3 4 5 6 7 8 9 10 11 12

In [5]:
def reduce_data_salem(data: xr.DataArray, shape: gpd.GeoDataFrame) -> xr.DataArray:
    reduced = data.salem.roi(shape=shape).mean(dim=["lat", "lon"])
    reduced = reduced.where(reduced > 0, 0)
    return reduced


clim_sectors = []
pisco_sectors = []
for sector_name in senamhi_sectors["sector"]:
    print(f"{sector_name=}")
    _sector = senamhi_sectors.query("sector == @sector_name")

    _reduced_obs = reduce_data_salem(pisco, _sector)
    _reduced_obs.name = sector_name

    _reduced_clim = reduce_data_salem(pisco_clim, _sector)
    _reduced_clim.name = sector_name

    clim_sectors.append(_reduced_clim)
    pisco_sectors.append(_reduced_obs)

clim_sectors = xr.combine_by_coords(clim_sectors)
obs_sectors = xr.combine_by_coords(pisco_sectors)

sector_name='CostaNorte'
sector_name='SierraNorteOccidental'
sector_name='CostaCentro'
sector_name='CostaSur'
sector_name='SierraCentralOccidental'
sector_name='SierraSurOccidental'
sector_name='SierraNorteOriental'
sector_name='SierraCentralOriental'
sector_name='SierraSurOriental'
sector_name='SelvaSurAlta'
sector_name='SelvaNorteAlta'
sector_name='SelvaCentralAlta'
sector_name='SelvaSurBaja'
sector_name='SelvaCentralBaja'
sector_name='SelvaNorteBaja'


In [6]:
pisco_sectores_df = obs_sectors.to_dataframe().applymap(
    lambda x: math.trunc(1000 * x) / 1000
)
pisco_clim_sectores = clim_sectors.to_dataframe().applymap(
    lambda x: math.trunc(1000 * x) / 1000
)

In [7]:
pisco_sectores_df["year"] = pisco_sectores_df.index.year
pisco_sectores_df["month"] = pisco_sectores_df.index.month
pisco_sectores_df = pisco_sectores_df.reset_index(drop=True)
pisco_sectores_df = pisco_sectores_df[
    ["year", "month"] + list(senamhi_sectors["sector"])
]

In [9]:
with pd.ExcelWriter("pisco_sectores.xlsx") as writer:
    pisco_sectores_df.to_excel(writer, sheet_name="PISCO", index=False)
    pisco_clim_sectores.to_excel(writer, sheet_name="CLIM")